<!-- NOTEBOOK_METADATA source: "Jupyter Notebook" title: "Observability for Pydantic AI with Langfuse Integration" description: "Discover how to integrate Langfuse with Pydantic AI for enhanced LLM application monitoring, debugging, and tracing. Improve your AI development workflow today." category: "Integrations" -->

# Integrate Langfuse with Pydantic AI

This notebook provides a step-by-step guide on integrating **Langfuse** with **Pydantic AI** to achieve observability and debugging for your LLM applications.

> **About PydanticAI:** [PydanticAI](https://pydantic-ai.readthedocs.io/en/latest/) is a Python agent framework designed to simplify the development of production-grade generative AI applications. It brings the same type-safety, ergonomic API design, and developer experience found in FastAPI to the world of GenAI app development. 

> **What is Langfuse?** [Langfuse](https://langfuse.com) is an open-source LLM engineering platform. It offers tracing and monitoring capabilities for AI applications. Langfuse helps developers debug, analyze, and optimize their AI systems by providing detailed insights and integrating with a wide array of tools and Pydantic AIs through native integrations, OpenTelemetry, and dedicated SDKs.

## Getting Started

Let's walk through a practical example of using Pydantic AI and integrating it with Langfuse for comprehensive tracing.

<!-- STEPS_START -->
### Step 1: Install Dependencies

<!-- CALLOUT_START type: "info" emoji: "⚠️" -->
_**Note:** This notebook utilizes the Langfuse OTel Python SDK v3. For users of Python SDK v2, please refer to our [legacy Pydantic AI integration guide](https://github.com/langfuse/langfuse-docs/blob/fdea27dd5d3f4a110a4f79c0ec1b7b381b48d091/cookbook/integration_pydantic_ai.ipynb)._
<!-- CALLOUT_END -->


In [ ]:
%pip install langfuse pydantic-ai -U

### Step 2: Configure Langfuse SDK

Next, set up your Langfuse API keys. You can get these keys by signing up for a free [Langfuse Cloud](https://cloud.langfuse.com/) account or by [self-hosting Langfuse](https://langfuse.com/self-hosting). These environment variables are essential for the Langfuse client to authenticate and send data to your Langfuse project.


In [1]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Your OpenAI key
os.environ["OPENAI_API_KEY"] = "sk-proj-..." 

With the environment variables set, we can now initialize the Langfuse client. `get_client()` initializes the Langfuse client using the credentials provided in the environment variables.


In [ ]:
from langfuse import get_client
 
langfuse = get_client()
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")


### Step 3: Initialize Pydantic AI Instrumentation

Now, we initialize the [Pydantic AI Instrumentation](https://ai.pydantic.dev/logfire/#logfire-with-an-alternative-otel-backend). This  automatically captures Pydantic AI operations and exports OpenTelemetry (OTel) spans to Langfuse.

In [3]:
from pydantic_ai.agent import Agent

# Initialize Pydantic AI instrumentation
Agent.instrument_all()

### Step 4: Basic Pydantic AI Application

Finally, run your Pydantic AI agent and generate trace data that will be sent to Langfuse. In the example below, the agent is executed with a dependency value (the winning square) and natural language input. The output from the tool function is then printed.

Make sure to pass `instrument=True` while configuring the `Agent`.

In [8]:
from pydantic_ai import Agent, RunContext

roulette_agent = Agent(
    'openai:gpt-4o',
    deps_type=int,
    result_type=bool,
    system_prompt=(
        'Use the `roulette_wheel` function to see if the '
        'customer has won based on the number they provide.'
    ),
    instrument=True
)

@roulette_agent.tool
async def roulette_wheel(ctx: RunContext[int], square: int) -> str:
    """check if the square is a winner"""
    return 'winner' if square == ctx.deps else 'loser'

In [ ]:
# Run the agent - using await since we're in a Jupyter notebook
success_number = 18
result = await roulette_agent.run('Put my money on square eighteen', deps=success_number)
print(result.output)

### Step 5: View Traces in Langfuse

After executing the application, navigate to your [Langfuse Trace Table](https://cloud.langfuse.com). You will find detailed traces of the application's execution, providing insights into the LLM calls, retrieval operations, inputs, outputs, and performance metrics. 

![Example Trace in Langfuse](https://langfuse.com/images/cookbook/otel-integration-pydantic-ai/pydanticai-openai-trace-tree.png)

[Example Trace in Langfuse](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/25f4bdeebaab60e6e1bee7e8469554bc?display=details%3Ftimestamp%3D2025-06-06T14%3A39%3A55.786Z%3Ftimestamp%3D2025-06-06T14%3A37%3A30.656Z%3Ftimestamp%3D2025-06-06T14%3A39%3A55.786Z?timestamp=2025-06-06T14%3A40%3A28.562Z)
<!-- STEPS_END -->

<!-- MARKDOWN_COMPONENT name: "LearnMore" path: "@/components-mdx/integration-learn-more.mdx" -->